In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
current_directory = os.getcwd()

In [3]:
current_directory

'C:\\Users\\nabar\\Documents\\Data_Science\\Challenge\\Zindi\\Digital Green Crop Yield Estimate Challenge'

In [4]:
# Load files
data_path = os.getcwd()
train = pd.read_csv(data_path + '/Train.csv')
test = pd.read_csv(data_path + '/Test.csv')
sample_submission = pd.read_csv(data_path + '/SampleSubmission.csv')
var_desc = pd.read_csv(data_path + '/VariableDescription.csv')

In [5]:
# Preview files
train.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,...,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
0,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-06-27,...,machine,2022-11-16,NaN,2022-11-16,machine,30,40,plowed_in_soil,0.312500,600
1,ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-06-20,...,hand,2022-11-25,3.0,2022-12-24,machine,24,10,plowed_in_soil,0.312500,600
2,ID_1FJY2CRIMLZZ,Gaya,Gurua,10,10,TractorPlough FourWheelTracRotavator,2022-06-30,6,Manual_PuddledRandom,2022-06-20,...,hand,2022-12-12,480.0,2023-01-11,machine,30,10,plowed_in_soil,0.148148,225
3,ID_I3IPXS4DB7NE,Gaya,Gurua,15,15,TractorPlough FourWheelTracRotavator,2022-06-16,6,Manual_PuddledRandom,2022-06-17,...,hand,2022-12-02,240.0,2022-12-29,hand,26,10,plowed_in_soil,0.222222,468
4,ID_4T8YQWXWHB4A,Nalanda,Noorsarai,60,60,TractorPlough WetTillagePuddling,2022-07-19,4,Manual_PuddledRandom,2022-06-21,...,machine,2022-11-30,NaN,2022-12-02,machine,24,40,plowed_in_soil,0.468750,550


In [6]:
test.head()

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,...,MineralFertAppMethod.1,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre
0,ID_F9XXEXN2ADR2,Jamui,Khaira,20,13,TractorPlough,2022-07-24,4,Manual_PuddledLine,2022-07-03,...,Broadcasting,hand,2022-12-02,500.0,2022-12-28,hand,22,10,plowed_in_soil,0.272727
1,ID_SO3VW2X4QO93,Jamui,Khaira,25,25,TractorPlough,2022-07-24,5,Manual_PuddledRandom,2022-07-08,...,Broadcasting,hand,2022-11-12,150.0,2022-12-20,machine,28,10,plowed_in_soil,0.227273
2,ID_UKUQ7JM8E894,Nalanda,Rajgir,30,30,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-07,5,Manual_PuddledRandom,2022-06-24,...,Broadcasting,hand,2022-11-24,1200.0,2022-12-05,machine,28,10,plowed_in_soil,0.281250
3,ID_QUISMWEZR2H4,Vaishali,Mahua,15,10,WetTillagePuddling TractorPlough BullockPlough...,2022-07-04,5,Manual_PuddledRandom,2022-06-20,...,SoilApplied,hand,2022-10-15,300.0,2022-10-25,hand,30,10,plowed_in_soil,0.136364
4,ID_25JGI455VKCZ,Nalanda,Rajgir,30,10,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-21,3,Manual_PuddledRandom,2022-06-24,...,Broadcasting,machine,2022-11-20,NaN,2022-11-20,machine,26,40,plowed_in_soil,0.312500


In [7]:
sample_submission.head()

,ID,Yield
0,ID_F9XXEXN2ADR2,0
1,ID_SO3VW2X4QO93,0
2,ID_UKUQ7JM8E894,0
3,ID_QUISMWEZR2H4,0
4,ID_25JGI455VKCZ,0


In [10]:
print('Size of Train', train.shape)
print('Size of Test', test.shape)

Size of Train (3870, 44)
Size of Test (1290, 43)


In [13]:
print("Number of duplicate rows Test:", train.duplicated().sum())
print("Number of duplicate rows Test:", test.duplicated().sum())

Number of duplicate rows Test: 0
Number of duplicate rows Test: 0


In [15]:
train.head(2)

,ID,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,...,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
0,ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-06-27,...,machine,2022-11-16,NaN,2022-11-16,machine,30,40,plowed_in_soil,0.3125,600
1,ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-06-20,...,hand,2022-11-25,3.0,2022-12-24,machine,24,10,plowed_in_soil,0.3125,600


In [14]:
#lets check the data types of the columns
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3870 entries, 0 to 3869
Data columns (total 44 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  3870 non-null   object 
 1   District                            3870 non-null   object 
 2   Block                               3870 non-null   object 
 3   CultLand                            3870 non-null   int64  
 4   CropCultLand                        3870 non-null   int64  
 5   LandPreparationMethod               3870 non-null   object 
 6   CropTillageDate                     3870 non-null   object 
 7   CropTillageDepth                    3870 non-null   int64  
 8   CropEstMethod                       3870 non-null   object 
 9   RcNursEstDate                       3787 non-null   object 
 10  SeedingSowingTransplanting          3870 non-null   object 
 11  SeedlingsPerPit                     3581 no

In [18]:
train = train[['District', 'Block', 'CropCultLand', 'LandPreparationMethod', 'CropEstMethod', 'RcNursEstDate', 'SeedingSowingTransplanting', 'SeedlingsPerPit', 'NursDetFactor', 'TransDetFactor', 'TransplantingIrrigationHours', 'StandingWater', 'OrgFertilizers', 'Ganaura', 'CropOrgFYM', 'PCropSolidOrgFertAppMethod', 'NoFertilizerAppln', 'CropbasalFerts', 'BasalDAP', 'BasalUrea', 'MineralFertAppMethod', 'FirstTopDressFert', '1tdUrea', '1appDaysUrea', '2tdUrea', '2appDaysUrea', 'MineralFertAppMethod', 'Harv_method', 'Harv_date', 'Residue_length', 'Residue_perc', 'Acre', 'Yield']]

In [19]:
# Check unique value for each attribute
train.nunique()

District                          4
Block                             9
CropCultLand                     91
LandPreparationMethod            43
CropEstMethod                     4
RcNursEstDate                    61
SeedingSowingTransplanting       62
SeedlingsPerPit                  21
NursDetFactor                   125
TransDetFactor                  155
TransplantingIrrigationHours     43
StandingWater                    14
OrgFertilizers                   31
Ganaura                          35
CropOrgFYM                       44
PCropSolidOrgFertAppMethod        4
NoFertilizerAppln                 4
CropbasalFerts                   34
BasalDAP                         44
BasalUrea                        41
MineralFertAppMethod              3
FirstTopDressFert                14
1tdUrea                          44
1appDaysUrea                     57
2tdUrea                          29
2appDaysUrea                     38
MineralFertAppMethod              3
Harv_method                 

In [20]:
#Check for missing value
train.isnull().sum()

District                           0
Block                              0
CropCultLand                       0
LandPreparationMethod              0
CropEstMethod                      0
RcNursEstDate                     83
SeedingSowingTransplanting         0
SeedlingsPerPit                  289
NursDetFactor                    289
TransDetFactor                   289
TransplantingIrrigationHours     193
StandingWater                    238
OrgFertilizers                  1335
Ganaura                         2417
CropOrgFYM                      2674
PCropSolidOrgFertAppMethod      1337
NoFertilizerAppln                  0
CropbasalFerts                   188
BasalDAP                         543
BasalUrea                       1704
MineralFertAppMethod               0
FirstTopDressFert                485
1tdUrea                          556
1appDaysUrea                     556
2tdUrea                         2694
2appDaysUrea                    2700
MineralFertAppMethod               0
H

In [ ]:
train['nurseryToTransplanting'] = train['SeedingSowingTransplanting'] - train['RcNursEstDate']

In [8]:
# Split data for training and local testing

X = train.drop(['ID', 'Yield'], axis = 1)
X = X.select_dtypes(include=np.number)
y = train.Yield

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1234)

# Instantiate model
model = RandomForestRegressor(random_state = 1234)

# Fit model
model.fit(X_train.fillna(0), y_train)

# Make predictions
preds = model.predict(X_test.fillna(0))

# Measure model performance
mean_squared_error(y_test, preds, squared=False)

393.8160470653371

In [9]:
# Make predictions on the Zindi test set
test_df = test[X.columns]
preds = model.predict(test_df.fillna(0))

# Create submisiion file to be uploaded to Zindi for scoring
sub = pd.DataFrame({'ID': test.ID, 'Yield': preds})
sub.to_csv('BenchmarkSubmission.csv', index = False)

sub.head()

,ID,Yield
0,ID_F9XXEXN2ADR2,671.40
1,ID_SO3VW2X4QO93,417.09
2,ID_UKUQ7JM8E894,493.95
3,ID_QUISMWEZR2H4,328.80
4,ID_25JGI455VKCZ,537.19
